# SPARQL 1.2

This document contains a short introduction to [RDF](https://www.w3.org/TR/rdf11-primer/) using [rudof](https://rudof-project.github.io/).


## Preliminaries: Install and configure rudof

The library is available as `pyrudof`.

In [1]:
!pip install pyrudof   # Pyrudof
! pip install ipython
! pip install plantuml # To visualize graphs
from IPython.display import Image
from pyrudof import Rudof, RudofConfig
rudof = Rudof(RudofConfig())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.7 MB/s eta 0:00:00


## Tim Berners-Lee exampl in RDF 1.2

Rudof supports [RDF 1.2](https://www.w3.org/TR/rdf12-primer).

RDF 1.2 introduces [triple terms](https://www.w3.org/TR/rdf12-primer/#section-triple-terms) which denote statements that can be the object of some triples.

The following example declares some information about a node `:timbl` that indicates that he worked at CERN between 1984 and 1994, and in 1980, and that he obtained the Princess of Asturias Award in 2002 together with Vinton Cerf,

In [10]:
rudof.read_data_str("""
prefix : <http://example.org/>
prefix sh:     <http://www.w3.org/ns/shacl#>
prefix xsd:    <http://www.w3.org/2001/XMLSchema#>
prefix rdfs:   <http://www.w3.org/2000/01/rdf-schema#>
prefix rdf:    <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

:timbl    rdfs:label    "Tim Berners Lee" .
_:r       rdf:reifies   <<( :timbl :employer :CERN )>> ;
          :start        "1984" ;
          :end          "1994" .
_:s       rdf:reifies   <<( :timbl :employer :CERN )>> ;
          :start        "1980" ;
          :end          "1980" .
_:t       rdf:reifies   <<( :timbl :award :PA )>> ;
          :time         "2002" ;
          :togetherWith :vint, :bob, :lawrence  .
:vint     rdfs:label    "Vinton Cerf" .
:bob      rdfs:label    "Bob kahn" .
:lawrence rdfs:label    "Lawrence Roberts" .
:PA       rdfs:label    "Princess of Asturias Award" .
:CERN     rdfs:label    "CERN" .
""")


## SPARQL 1.2

In [11]:
results = rudof.run_query_str("""
prefix : <http://example.org/>
prefix rdfs:   <http://www.w3.org/2000/01/rdf-schema#>
prefix rdf:    <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?person ?employer ?start ?end WHERE {
  ?r rdf:reifies <<( ?person :employer ?employer )>> ;
     :start ?start ;
     :end ?end .
 }
 """)
print(results.show())


╭───┬─────────┬───────────┬────────┬────────╮
│   │ ?person │ ?employer │ ?start │ ?end   │
├───┼─────────┼───────────┼────────┼────────┤
│ 1 │ :timbl  │ :CERN     │ "1984" │ "1994" │
├───┼─────────┼───────────┼────────┼────────┤
│ 2 │ :timbl  │ :CERN     │ "1980" │ "1980" │
╰───┴─────────┴───────────┴────────┴────────╯



In [14]:
results = rudof.run_query_str("""
prefix : <http://example.org/>
prefix rdfs:   <http://www.w3.org/2000/01/rdf-schema#>
prefix rdf:    <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?person ?award ?togetherPerson ?time WHERE {
  ?r rdf:reifies <<( ?person :award ?award )>> ;
  OPTIONAL {?r :time ?time }
  OPTIONAL {
    ?r            :togetherWith ?togetherWith .
    ?togetherWith rdfs:label    ?togetherPerson
  }
 }
 """)
print(results.show())

╭───┬─────────┬────────┬────────────────────┬────────╮
│   │ ?person │ ?award │ ?togetherPerson    │ ?time  │
├───┼─────────┼────────┼────────────────────┼────────┤
│ 1 │ :timbl  │ :PA    │ "Vinton Cerf"      │ "2002" │
├───┼─────────┼────────┼────────────────────┼────────┤
│ 2 │ :timbl  │ :PA    │ "Lawrence Roberts" │ "2002" │
├───┼─────────┼────────┼────────────────────┼────────┤
│ 3 │ :timbl  │ :PA    │ "Bob kahn"         │ "2002" │
╰───┴─────────┴────────┴────────────────────┴────────╯

